In [3]:
# use env:gdal_env_2.7v.2
import geopandas as gpd
import rasterio
from rasterio import features
import numpy as np

Set up your filenames

In [4]:
shp_fn = r'../../ts-raster-example-data/fire17_1/fire17_1.shp'
rst_fn = r'../../ts-raster-example-data/fire17_1/example.tif'
out_fn =  r'../../ts-raster-example-data/fire17_1/AnnualRasters/fire_1980.tif'

Open the file with GeoPANDAS read_file

In [5]:
fires = gpd.read_file(shp_fn)

CHeck and fix datatype issues

In [ ]:
#print fires.columns
#fires['YEAR_'].head()
#fires.YEAR_.unique()

In [6]:
print fires.dtypes.head()

YEAR_        object
STATE        object
AGENCY       object
UNIT_ID      object
FIRE_NAME    object
dtype: object


In [7]:
# subset to year and convert to integer
fires = fires[fires['YEAR_'].isin(['1980'])]
fires['YEAR_'] = fires['YEAR_'].astype(str).astype(int)
print fires.dtypes.head()

YEAR_         int32
STATE        object
AGENCY       object
UNIT_ID      object
FIRE_NAME    object
dtype: object


Open the raster file you want to use as a template for feature burning using rasterio


In [8]:
rst = rasterio.open(rst_fn)
rst.bounds

BoundingBox(left=-374495.83635354, bottom=-616363.3341887, right=566504.16364646, top=592636.6658113)

copy and update the metadata from the input raster for the output


In [23]:
meta = rst.meta.copy()
meta.update(compress='lzw')

Now burn the features into the raster and write it out


In [24]:
with rasterio.open('C:/Users/mmann/Documents/ts-raster-example-data/fire17_1/example.tif', 'w', **meta) as out:
    out_arr = out.read()

    # this is where we create a generator of geom, value pairs to use in rasterizing
    shapes = ((geom,value) for geom, value in zip(fires.geometry, fires.YEAR_))

    burned = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=out.transform)
    out.write_band(1, burned)



UnsupportedOperation: not readable

In [12]:
# Read raster bands directly to Numpy arrays.
with rasterio.open(rst_fn) as src:
    out_arr = src.read()

In [21]:
# read in example raster
rst = rasterio.open(rst_fn)

# Write the product as a raster band to a new 8-bit file. For
# the new file's profile, we start with the meta attributes of
# the source file, but then change the band count to 1, set the
# dtype to uint8, and specify LZW compression, missing = 0.
profile = rst.profile
profile.update(dtype=rasterio.uint8, count=1, compress='lzw',nodata=0)
profile

with rasterio.open(rst_fn, 'w', **profile) as dst:
    out_arr = dst.read()
    
    # this is where we create a generator of geom, value pairs to use in rasterizing
    shapes = ((geom,value) for geom, value in zip(fires.geometry, fires.YEAR_))

    burned = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=out.transform)
    out.write(total.astype(rasterio.uint8), 1)
    
#with rasterio.open('example-total.tif', 'w', **profile) as dst:
  #  dst.write(total.astype(rasterio.uint8), 1)

UnsupportedOperation: not readable

The overall idea is to create an iterable containing tuples of (geometry, value), where the geometry is a shapely geometry and the value is what you want to burn into the raster at that geometry's location. Both Fiona and GeoPANDAS use shapely geometries so you are in luck there. In this example a generator is used to iterate through the (geometry,value) pairs which were extracted from the GeoDataFrame and joined together using zip().

In [22]:
 rst_fn

'../../ts-raster-example-data/fire17_1/example.tif'

 rst_fn